1. PREREQUISITES

In [ ]:
""" 
1 Install Kubectl

#https://kubernetes.io/docs/tasks/tools/install-kubectl-macos/#install-kubectl-on-macos

curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/darwin/amd64/kubectl"

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin/kubectl
sudo chown root: /usr/local/bin/kubectl

kubectl version --client

"""

In [ ]:
"""
2 Install Kops

https://kops.sigs.k8s.io/getting_started/install/

# #brew install kops

"""

In [ ]:
"""
# 3 Create bucket 

gsutil mb gs://kubernetes-clusters-mlendtoend

"""

2. KUBERNETES CLUSTER creation using KOPS

In [ ]:
"""

export KOPS_STATE_STORE=gs://kubernetes-clusters-mlendtoend/
kops create cluster # creates the Cluster object and InstanceGroup object you'll be working with in kOps:
kops create cluster simple.k8s.local --zones asia-south1-a --state ${KOPS_STATE_STORE}/ --project=${PROJECT}

## NOTE - > 
# Change kubernetes version to 1.26.12 to be compatible with kubeflow
# Change machine type as per reuriement

Suggestions:
 * list clusters with: kops get cluster
 * edit this cluster with: kops edit cluster simple.k8s.local
 * edit your node instance group: kops edit ig --name=simple.k8s.local nodes-asia-south1-a
 * edit your control-plane instance group: kops edit ig --name=simple.k8s.local control-plane-asia-south1-a

Finally configure your cluster with: kops update cluster --name simple.k8s.local --yes --admin

kops update cluster simple.k8s.local
kops update cluster simple.k8s.local --yes



Suggestions:
 * validate cluster: kops validate cluster --wait 10m
 * list nodes: kubectl get nodes --show-labels
 * ssh to a control-plane node: ssh -i ~/.ssh/id_rsa ubuntu@



# Export the config using below
kops export kubeconfig --admin


kubectl get nodes

kubectl version --short


"""

In [ ]:
"""
#To delete

kops delete cluster simple.k8s.local --yes

"""